# STOCK MARKET SHORT PREDICTION MODEL

@author: Matheus José Oliveira dos Santos

## 1. Imports

In [ ]:
import sys
import os
from datetime import date, datetime

sys.path.append(os.getcwd()+'\\..\\src')
import db_interface
import portfolio_backtest